<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; text-align: left; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Jan Attig</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; font-size: 30px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; font-size: 30px; padding:0px;">Übungsblatt 4</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; font-size: 20px; padding:0px; margin-bottom: 20px;">Sommersemester 2020</h3>
<hr>

**Website** [http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml)

**Abgabe**:  18.5.2020 bis 13:00 Uhr

**Besprechung**:  20.5.2020 - 22.5.2020

**Name**: <font color="red">Bitte geben Sie ihren Namen an</font>

**Matrikelnummer**: <font color="red">Bitte geben Sie ihre Matrikelnummer an</font>

<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; text-align: left; font-size: 25px; font-weight: bold; text-decoration: underline">Aufgaben auf Übungsblatt 4</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 15px"></td></tr></table>

- Simpson vs. Trapezregel [6 Punkte]
- Statistik einer Epidemie [14 Punkte]
- **(Bonus) Zweidimensionale Zelluläre Automaten** [10 Bonus-Punkte]


<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; text-align: left; font-size: 25px; font-weight: bold; text-decoration: underline">Zweidimensionale zelluläre Automaten</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 15px">[10 Bonus-Punkte]</td></tr></table>

In dieser *optionalen Aufgabe* betrachten wir die zeitlichen Entwicklung zweier **zweidimensionaler zellulärer Automaten**. Ein solcher zellulärer Automat ist ein diskretes System aus einzelnen Zellen, die in einem Quadratgitter angeordnet sind und ausgehend von ihrem eigenen Zustand und dem der umgegebenen Zellen in jedem Zeitschritt aktualisiert werden.

In dieser Aufgabe wollen wir uns auf zwei verschiedene Systeme konzentrieren. Zum einen behandeln wir einen *chemischen zellulären Automaten* mit drei Parametern $a, b, c \in [0, 1)$ und zum anderen das bekannte [Game of Life](https://de.wikipedia.org/wiki/Conways_Spiel_des_Lebens) von John Conway. Wir lassen diese Aufgabe auch deshalb in diesem Semester laufen, weil [John Conway](https://de.wikipedia.org/wiki/John_Horton_Conway) – einer der originellsten Mathematiker seiner Generation – bedauerlicherweise Anfang April verstorben ist; eine sehr schöne Beschreibung seines Wirkens findet sich in diesem 2015 erschienen [Artikel im Guardian](https://www.theguardian.com/science/2015/jul/23/john-horton-conway-the-most-charismatic-mathematician-in-the-world), sowie in den Nachrufen der [New York Times](https://nyti.ms/3epJ7dz) und des [Quanta Magazines](https://www.quantamagazine.org/john-conway-solved-mathematical-problems-with-his-bare-hands-20200420/).

Um Ihnen den Einstieg in diese Aufgabe zu erleichern, stellen wir Ihnen die nötigen Funktionen zur Seite, die Ihren Automaten animieren werden (basierend auf Ihrer Berechnung der Zustände). Beachten Sie jedoch bitte, dass diese Funktionen nicht in Julia Box funktionieren.

In [ ]:
# Useful Base Libraries
using LinearAlgebra
using Statistics

# Plotting Stuff
using PyPlot # plotting library
pygui(true)  # external window

# generieren der Figur
function generate_image(data; zoom=1.0)
    fig = figure(figsize=size(data), frameon=false, dpi=1*zoom)
    image = imshow(data', interpolation="none", origin="lower")
    axis("off")
    fig.subplots_adjust(bottom=0, top=1, right=1, left=0)
    return image
end

# Updaten der Figur
function update_image!(image, data)
    image.set_array(data')
    pause(0.001)
end

Um Ihnen zudem einen einfachen Einstieg in die Programmierung von zellulären Automaten zu geben, stellen wir Ihnen zusätzlich eine Funktion `neighbors` bereit, die die Summe der Nachbarn um eine Stelle `i,j` in einem gegebenen Array (unter Berücksichtigung von *periodischen Randbedingungen*) berechnet.

In [ ]:
# Summe der Nachbarn um (i,j) im Array data berechnen
function neighbors(data, i, j)
    if i == size(data,1)
        ip1 = 1
        im1 = i-1
    elseif i == 1
        ip1 = i+1
        im1 = size(data,1)
    else
        ip1 = i+1      
        im1 = i-1 
    end
    if j == size(data,2)
        jp1 = 1
        jm1 = j-1
    elseif j == 1
        jp1 = j+1
        jm1 = size(data,2)
    else
        jp1 = j+1
        jm1 = j-1
    end
    return  data[ip1, j] +
            data[im1, j] +
            data[ip1, jp1] +
            data[im1, jp1] +
            data[ip1, jm1] +
            data[im1, jm1] +
            data[i, jp1] +
            data[i, jm1]
end

### Teil A - Chemischer zellulärer Automat

Bei diesem Automaten werden die Zellen durch drei Parameter $a, b, c \in [0, 1)$ beschrieben werden, welche die Konzentration dreier Chemikalien beschreiben sollen. Das Aktualisieren der Werte soll chemische Reaktionen zwischen diesen Substanzen beschreiben. Zu Anfang seien die drei Konzentrationen zufällig aus dem möglichen Wertebereich gewählt.

Der zelluläre Automat funktioniert nun nach den folgenden Regeln:
* Für jede Zelle wird der durchschnittliche Wert $\overline{a}, \overline{b}, \overline{c}$ als Mittelwert der acht umgebenden Zellen sowie des Werts der Zelle selbst berechnet.
* Der neue Zustand ist dann gegeben durch

$\quad a = \min\left(1.0,\; \overline{a}\cdot\left(1. + \overline{b} - \overline{c}\right)\right)$

$\quad b = \min\left(1.0,\; \overline{b}\cdot \left(1. + \overline{c} - \overline{a}\right)\right)$

$\quad c = \min\left(1.0,\; \overline{c}\cdot \left(1. + \overline{a} - \overline{b}\right)\right)$

Ihre Aufgabe ist es das Programm-Skelett in den nachfolgenen Zellen zu vervollständigen und um das Update der Automaten-Zellen zu erweitern. Wie Sie erkennen können, wird der Automat bereits visualisiert indem eine der drei Konzentrationen dargestellt und in jedem Schritt aktualisiert wird. 

Es kann hilfreich sein, die Funktion `neighbors` zu benutzen, die oben bereits definiert ist. Diese erzeugt die Summe der umliegenden Nachbarn in einem gegebenen Array an einer gegebenen Position.

Beachten Sie schließlich, dass alle Zellen *gleichzeitig* in den neuen Zustand übergehen, d.h. die Berechnung der Mittelwerte darf ausschließlich mit den Werten des vorherigen Iterationsschritts passieren und nicht mit denen der schon partiell veränderten Matrix.


In [ ]:
# Systemgroesse
Lx = 200
Ly = 100

# Die Werte a,b,c des Automaten als 3 Arrays der Groesse LxL
automaton_a = rand(Lx, Ly)
automaton_b = rand(Lx, Ly)
automaton_c = rand(Lx, Ly)

# Arrays fuer die Mittelwerte
automaton_mean_a = zeros(Lx, Ly)
automaton_mean_b = zeros(Lx, Ly)
automaton_mean_c = zeros(Lx, Ly)

# Erstes darstellen des Bildes
image = generate_image(automaton_a, zoom=4)


# Fuehre Schritte aus
for step in 1:200
    
    ############################################
    # HIER SOLLTE IHR CODE STEHEN
    # Berechne die Mittelwerte    
    # Berechne die neuen Automatenzustaende
    ############################################

    # Aktualisiere das Bild
    update_image!(image, automaton_a)
end

### Teil B - Conway's Game of Life

Im Folgenden wollen wir nun einen weiteren zellulären Automaten betrachten, der als [Conway's Game of Life](https://de.wikipedia.org/wiki/Conways_Spiel_des_Lebens) bekannt wurde. Die Grundlage dieses Automaten sind erneut Zellen, die in einem Quadratgitter angeordnet sind. Im Unterschied zum biologischen Automaten können diese jedoch nur zwischen zwei Zuständen wechseln, die wir im Folgenden als tot (`0`) und lebendig (`1`) bezeichnen wollen.

In jedem Iterationsschritt wird nun für jede Zelle geprüft, ob sie lebt oder tot ist und wie viele lebendige Nachbarn sie besitzt. Daraus wird entwickelt, wie sich die Zelle im nächsten Iterationsschritt verhält:
- Eine Zelle mit weniger als 2 lebendigen Nachbarn ist sicher im nächsten Schritt tot
- Eine tote Zelle mit exakt 3 lebendigen Nachbarn wird im nächsten Schritt geboren
- Eine lebeindige Zelle mit mehr als 3 lebendigen Nachbarn stirbt in der nächsten Generation an Überbevölkerung

Ihre Aufgabe ist die Implementierung des Automaten basierend auf dem Code aus dem vorherigen Aufgabenteil. Testen Sie Ihre Implementierung indem Sie ein zufällig besetztes Array generieren und dieses als Startkonfiguration benutzen.

In [ ]:
# Systemgroesse
Lx = 1280
Ly = 720

############################################
# HIER SOLLTE IHR CODE STEHEN
############################################

Wie Sie sicherlich erkennen konnten, entwickelt sich die zufällige Konfiguration in vielerlei Hinsicht. Es gibt kleine Strukturen und Cluster die scheinbar statisch scheinen, sowie Elemente, die kurze Perioden durchlaufen (sogenannte Pulsare). Die Menge an verschiedene Konfigurationen und Mustern ist gigantisch und kann im [Game of Life Wiki](http://www.conwaylife.com/wiki/Main_Page) nachgelesen werden. Wenn sie dort verschiedenen Konfigurationen, wie die [Gosper Gleiterkanone](http://www.conwaylife.com/wiki/Gosper_glider_gun) o.Ä. betrachten, können Sie diese auch in einem sogenannten [RLE](http://www.conwaylife.com/wiki/RLE) Format direkt kopieren.

Probieren Sie Ihren Code um eine Funktion zu erweitern, die aus einen RLE String in ihre Simulation einlesen kann. Welches Muster finden Sie am interessantesten?

Hinweis: Sie mögen das String Macro `raw"..."` hier nützlich finden, da es Strings mit enthaltenen `$` darstellen kann. Z.B. erzeugt `raw"ob$b"` wirklich den entsprechenden String und will nicht den Buchstaben `b` interpretieren.